In [ ]:
!pip install torchtext==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2


In [ ]:
import torch
from torchtext import data
import torchtext

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Text = data.Field(tokenizer_language='en_core_web_sm',
                   include_lengths=True,
                   tokenize='spacy')
Label=data.LabelField(dtype=torch.float)

In [ ]:
from torchtext import datasets
train_data , test_data = datasets.IMDB.splits(Text,Label)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 33.7MB/s]


In [ ]:
import random

train_data,valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
MAX_VOCAB = 25_000

Text.build_vocab(train_data,max_size=MAX_VOCAB,
                vectors = 'glove.6B.100d',
                unk_init = torch.Tensor.normal_)
Label.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19918.86it/s]


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator , test_iterator , valid_iterator = data.BucketIterator.splits(
    (train_data,test_data,valid_data),
    batch_size= BATCH_SIZE,
    device = device,
    sort_within_batch = True
)

#model

In [ ]:
from torch import nn
class RNN(nn.Module):
    def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim,num_layers,bidirectional,dropout,pad_idx):
        super(RNN,self).__init__()

        self.embedding=nn.Embedding(input_dim,embedding_dim,padding_idx = pad_idx)

        self.lstm = nn.LSTM(embedding_dim,hidden_dim,num_layers=num_layers,dropout=dropout,bidirectional=bidirectional)

        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(hidden_dim *2 ,output_dim)

    def forward(self,text,text_length):

        embedding = self.dropout(self.embedding(text))

        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedding,text_length.to('cpu'))

        packed_output , (hidden,cell) = self.lstm(packed_embedding)

        output , output_length = nn.utils.rnn.pad_packed_sequence(packed_output)

        hidden = self.dropout(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1))

        return self.fc(hidden)

In [ ]:
input_dim=len(Text.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1
num_layers=2
bidirectional = True
dropout = 0.5
pad_idx = Text.vocab.stoi[Text.pad_token]

model = RNN(input_dim,embedding_dim,hidden_dim,output_dim,num_layers,bidirectional,dropout,pad_idx)

In [ ]:
pretrained_weights = Text.vocab.vectors

In [ ]:
model.embedding.weight.data.copy_(pretrained_weights)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4337,  1.4285,  1.0520,  ..., -0.0274, -0.8621,  0.8455],
        [ 0.8448, -0.5013, -0.5840,  ...,  0.1304, -0.1945, -0.0420],
        [-0.4625, -0.1248, -0.3915,  ..., -0.7118,  0.0469,  0.4989]])

In [ ]:
unk_idx = Text.vocab.stoi[Text.unk_token]

model.embedding.weight.data[pad_idx]=torch.zeros(embedding_dim)
model.embedding.weight.data[unk_idx]=torch.zeros(embedding_dim)

#config

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def binary_accuracy(pred,y):

    rounded_preds = torch.round(torch.sigmoid(pred))
    corrects = (y==rounded_preds).float()
    acc = sum(corrects) / len(corrects)
    return acc

In [ ]:
binary_accuracy(torch.tensor([1,0,0,1]),torch.tensor([1,1,0,0])).item()

0.5

In [ ]:
model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

In [ ]:
def train(model,iterator,criterion,optimizer):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()
        text,text_length = batch.text

        pred=model(text,text_length).squeeze(1)

        loss = criterion(pred,batch.label)

        epoch_loss += loss.item()
        epoch_acc += binary_accuracy(pred,batch.label).item()

        loss.backward()
        optimizer.step()

    return epoch_loss/len(iterator) , epoch_acc/len(iterator)

In [ ]:
def evaluation(model,iterator,criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text,text_length = batch.text

            pred=model(text,text_length).squeeze(1)
            loss = criterion(pred,batch.label)
            acc=binary_accuracy(pred,batch.label)

            epoch_loss += loss.item()
            epoch_acc +=acc.item()


        return epoch_loss/len(iterator) , epoch_acc/len(iterator)

In [ ]:
import time

def epoch_time(start_time,end_time):

    elapsed_time = end_time - start_time
    min_elapsed = int(elapsed_time/60)
    sec_elapsed = int(elapsed_time-min_elapsed*60)

    return min_elapsed , sec_elapsed

In [ ]:
N_EPOCH = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCH):
    start_time = time.time()

    train_loss,train_acc = train(model,train_iterator,criterion,optimizer)
    valid_loss , valid_acc = evaluation(model,valid_iterator,criterion)

    end_time = time.time()

    elapsed_min,elapsed_sec = epoch_time(start_time,end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(),'model-tut3.pt')

    print(f'Epoch {epoch+1:02} Epoch Time: {elapsed_min}m , {elapsed_sec}s')
    print(f'\tTrain Loss: {train_loss:.3f} , Train Acc:{train_acc*100:.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} , Valid Acc:{valid_acc*100:.2f}%')

Epoch 01 Epoch Time: 0m , 36s
	Train Loss: 0.659 , Train Acc:60.36%
	Valid Loss: 0.606 , Valid Acc:62.05%
Epoch 02 Epoch Time: 0m , 36s
	Train Loss: 0.625 , Train Acc:64.38%
	Valid Loss: 0.462 , Valid Acc:79.75%
Epoch 03 Epoch Time: 0m , 36s
	Train Loss: 0.530 , Train Acc:74.35%
	Valid Loss: 0.420 , Valid Acc:83.49%
Epoch 04 Epoch Time: 0m , 36s
	Train Loss: 0.395 , Train Acc:82.94%
	Valid Loss: 0.344 , Valid Acc:86.81%
Epoch 05 Epoch Time: 0m , 36s
	Train Loss: 0.317 , Train Acc:86.95%
	Valid Loss: 0.291 , Valid Acc:88.26%


In [ ]:
model.load_state_dict(torch.load('model-tut3.pt'))

test_loss , test_acc = evaluation(model,test_iterator,criterion)
print(f'\tTest Loss: {test_loss:.3f} , Test Acc:{test_acc*100:.2f}%')

	Test Loss: 0.309 , Test Acc:87.07%
